In [2]:
import tensorflow as tf
import numpy as np

In [3]:
a = np.array(np.arange(1,28), dtype=np.float32).reshape([1,3,3,3])
print(a)

[[[[ 1.  2.  3.]
   [ 4.  5.  6.]
   [ 7.  8.  9.]]

  [[10. 11. 12.]
   [13. 14. 15.]
   [16. 17. 18.]]

  [[19. 20. 21.]
   [22. 23. 24.]
   [25. 26. 27.]]]]


上面的代码构建了1个3x3通道为3的图片数据，3个通道的数值分别为

| - | - | - |
|---|---|---|
| 1 | 4 | 7 |
| 10| 13| 16|
| 19| 22| 25|

| - | - | - |
|---|---|---|
| 2 | 5 | 8 |
| 11| 14| 17|
| 20| 23| 26|

| - | - | - |
|---|---|---|
| 3 | 6 | 9 |
| 12| 15| 18|
| 21| 24| 27|

In [4]:
kernel = np.array(np.arange(1,61), dtype=np.float32).reshape([2,2,3,5])
print(kernel)

[[[[ 1.  2.  3.  4.  5.]
   [ 6.  7.  8.  9. 10.]
   [11. 12. 13. 14. 15.]]

  [[16. 17. 18. 19. 20.]
   [21. 22. 23. 24. 25.]
   [26. 27. 28. 29. 30.]]]


 [[[31. 32. 33. 34. 35.]
   [36. 37. 38. 39. 40.]
   [41. 42. 43. 44. 45.]]

  [[46. 47. 48. 49. 50.]
   [51. 52. 53. 54. 55.]
   [56. 57. 58. 59. 60.]]]]


上面的代码构建了5x3个2x2的矩阵，如下：

#### 第一组（对应input的3个channels）

|-|-|
|-|-|
|1|16|
|31|46|

|-|-|
|-|-|
|6|21|
|36|51|

|-|-|
|-|-|
|11|26|
|41|56|

#### 第二组（对应input的3个channels）

|-|-|
|-|-|
|2|17|
|32|47|

|-|-|
|-|-|
|7|22|
|37|52|

|-|-|
|-|-|
|12|27|
|42|57|

#### 第三组（对应input的3个channels）

|-|-|
|-|-|
|3|18|
|33|48|

|-|-|
|-|-|
|8|23|
|38|53|

|-|-|
|-|-|
|13|28|
|43|58|

#### 第四组（对应input的3个channels）

|-|-|
|-|-|
|4|19|
|34|49|

|-|-|
|-|-|
|9|24|
|39|54|

|-|-|
|-|-|
|14|29|
|44|59|

#### 第五组（对应input的3个channels）

|-|-|
|-|-|
|5|20|
|35|50|

|-|-|
|-|-|
|10|25|
|40|55|

|-|-|
|-|-|
|15|30|
|45|60|

**从上面可以看出，kernel卷积的大小（2x2）和input没有关系，是自己定义的。第3个参数input_channels=3为什么一定要和输入数据的channels一致呢？因为在我们需要从这层每次提取一个元素来组建上面的一组卷积矩阵，output_channels=5表示要准备好多少组这样的卷积矩阵，每组卷积矩阵学习一个特征，5表示希望学习到5种特征**

In [10]:
# Must have strides[0] = strides[3] = 1
conv2d = tf.nn.conv2d(a, kernel, strides=[1,1,1,1], padding='VALID')

In [16]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    result = sess.run(conv2d)
    print(result)

[[[[3721. 3817. 3913. 4009. 4105.]
   [4747. 4879. 5011. 5143. 5275.]]

  [[6799. 7003. 7207. 7411. 7615.]
   [7825. 8065. 8305. 8545. 8785.]]]]


**输出Channel-1(特征1)：**

|-|-|
|-|-|
|3721|4747|
|6799|7825|

3721 = (1\*1+16\*4+31\*10+46\*13) + (6\*2+21\*5+36\*11+51\*14) + (11\*3+26\*6+41\*12+56\*15)

4747 = (1\*4+16\*7+31\*13+46\*16) + (6\*5+21\*8+36\*14+51\*17) + (11\*6+26\*9+41\*15+56\*18)

6799 = (1\*10+16\*13+31\*19+46\*22) + (6\*11+21\*14+36\*20+51\*23) + (11\*12+26\*15+41\*21+56\*24)

7825 = (1\*13+16\*16+31\*22+46\*25) + (6\*14+21\*17+36\*23+51\*26) + (11\*15+26\*18+41\*24+56\*27)

说明conv2d每个channel之间的卷积结果是相加的，则整个计算过程为：点乘、加、加

**输出Channel-2(特征2)：**

|-|-|
|-|-|
|3817|4879|
|7003|8065|

3817 = (2\*1+17\*4+32\*10+47\*13) + (7\*2+22\*5+37\*11+52\*14) + (12\*3+27\*6+42\*12+57\*15)

4879 = (2\*4+17\*7+32\*13+47\*16) + (7\*5+22\*8+37\*14+52\*17) + (12\*6+27\*9+42\*15+57\*18)

7003 = (2\*10+17\*13+32\*19+47\*22) + (7\*11+22\*14+37\*20+52\*23) + (12\*12+27\*15+42\*21+57\*24)

8065 = (2\*13+17\*16+32\*22+47\*25) + (7\*14+22\*17+37\*23+52\*26) + (12\*15+27\*18+42\*24+57\*27)

**其它输出层（特征提取）的计算过程类似，自己推到就好。**
